In [1]:
'''pip install nbimporter
'''

'pip install nbimporter\n'

In [2]:
import nbimporter

#==============================Dependencies================================

#libraries
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, truncnorm
from numpy.random import Generator, MT19937
import time

#Initializing the random number generator
seed = int(time.time()) 
bitgen = MT19937(seed)
rng = Generator(bitgen)  # reproducible generator

In [3]:
from hybrid_mcmc_KDE import metropolis_python, silverman_bandwidth, adaptive_kde, hybrid_importance_sampling  

from Importance_Analysis import importance_integration, target_pdf

from Uniform_MC import uniform_mc

ModuleNotFoundError: No module named 'Importance_Analysis'

In [ ]:
def f(x):
    return 1 / (1 + x**2)  # Cauchy distribution

In [ ]:
m_chain = metropolis_python(f, N, rng.uniform(-1, 1), 4, 200, 1)
h_fixed = silverman_bandwidth(m_chain)
kde = adaptive_kde(m_chain, h_fixed, alpha=0.5)

print("Hybrid Importance Sampling Estimate:"),
hybrid_importance_sampling(f, m_chain, kde)

Acceptance rate: 0.45
Hybrid Importance Sampling Estimate:


(3.1214641475962237, 2.744220087376017e-05)